In [ ]:
# prueba del transformer
import numpy as np
import torch  # libreria principal de python
import torch.nn as nn  # modulo para las redes neuronales
import torch.optim as optim  # modulo para algoritmos de optimizacion en redes neuronales
import torch.utils.data as data  # modulo para tratar con los datasets
import math  # operaciones matematicas
import copy  # para copiar objetos y estructuras

import pandas as pd
from Tokenizador.Tokenizador import TokenizadorBatch

In [ ]:
# cargamos el modelo completo
transformer = torch.load("transformer_modelo_completo.pth", map_location=torch.device("cpu"))
transformer.eval()

In [ ]:
import torch

device = torch.device("cpu")  # o tu device
#transformer.eval()  # modo evaluación

# Inicializamos tu tokenizador
tokenizador = TokenizadorBatch(max_length=64, batch_size=1, device=device)

# Frase de prueba
frase_test = "Llamame temprano"

# ---------------------------
# 1. Tokenizar entrada para el encoder
# ---------------------------
entrada_encoder, _ = tokenizador.tokenizar_fila(frase_test, frase_test)
entrada_encoder = entrada_encoder.squeeze(0).unsqueeze(0)  # [1, seq_len]

# ---------------------------
# 2. Inicializar decoder con token de inicio
# ---------------------------
bos_token_id = tokenizador.modelo_tokenizador.bos_token_id
eos_token_id = tokenizador.modelo_tokenizador.eos_token_id
dec_input = torch.tensor([[bos_token_id]], device=device)  # [1, 1]

# ---------------------------
# 3. Generación paso a paso
# ---------------------------
max_len_dec = 20
salida_ids = []

for _ in range(max_len_dec):
    with torch.no_grad():
        # Forward del transformer
        output = transformer(entrada_encoder, dec_input)  # [batch, seq_len, vocab_size]
        
        # Tomar el último token predicho
        next_token = output[:, -1, :].argmax(-1).item()
        salida_ids.append(next_token)
        
        # Concatenar el token predicho al decoder
        dec_input = torch.cat([dec_input, torch.tensor([[next_token]], device=device)], dim=1)
        
        # Si predice EOS, terminamos
        if next_token == eos_token_id:
            break

# ---------------------------
# 4. Decodificar los IDs a texto
# ---------------------------
prediccion = tokenizador.modelo_tokenizador.decode(salida_ids, skip_special_tokens=True)

print("Entrada:", frase_test)
print("Predicción:", prediccion)
